In [ ]:
# Aggregate data by date, inserting the mean of each day in the 'actual' column
dataset_agg = dataset.groupby('date_c').agg({'actual': 'mean', 'date_c': 'count'})
# Rename columns
dataset_agg = dataset_agg.rename(columns={'actual': 'daily_mean', 'date_c': 'count'})
# Create the deviation column
dataset_agg['deviation'] = dataset_agg['daily_mean'] - qc_rows['nominal'].iloc[0]

In [ ]:
# Convert the index of both DataFrames to DatetimeIndex
df_pre_2024.index = pd.to_datetime(df_pre_2024['date_c'])
df_2024.index = pd.to_datetime(df_2024['date_c'])

# Create Pandas Series for the detrended data, using the DataFrames' DatetimeIndex
detrended_pre_series = pd.Series(detrended_pre, index=df_pre_2024.index)
detrended_2024_series = pd.Series(detrended_2024, index=df_2024.index)

# Resample the detrended Series to daily frequency and interpolate missing values
int_detrended_pre = detrended_pre_series.resample('D').interpolate()
int_detrended_2024 = detrended_2024_series.resample('D').interpolate()

# Extract the values from the resampled Series as NumPy arrays
x_pre = int_detrended_pre.values
x_2024 = int_detrended_2024.values

# Define the sampling frequency (1 sample per day)
fs = 1

# Calculate the periodogram for the resampled data
f_pre, p_pre = signal.periodogram(x_pre, fs)
f_2024, p_2024 = signal.periodogram(x_2024, fs)

# Create the figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=f_pre, y=p_pre, mode='lines', name='Periodogram pre-2024'))
fig.add_trace(go.Scatter(x=f_2024, y=p_2024, mode='lines', name='Periodogram 2024'))
fig.update_layout(title='Periodogram of Detrended Interpolated Data',
                  xaxis_title='Frequency (cycles/day)',
                  yaxis_title='Power Spectral Density')
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
from datetime import datetime, timedelta
import random

# 1. Generazione di date e orari casuali non uniformi
def generate_random_times(n_points, start_date, end_date):
    total_seconds = int((end_date - start_date).total_seconds())
    random_seconds = sorted(random.sample(range(total_seconds), n_points))
    datetimes = [start_date + timedelta(seconds=sec) for sec in random_seconds]
    return datetimes

# Parametri
n_points = 100
start = datetime(2023, 1, 1)
end = datetime(2023, 3, 1)
datetimes = generate_random_times(n_points, start, end)

# 2. Conversione dei datetime in giorni (float) per Lomb-Scargle
ref_time = datetimes[0]
times = np.array([(dt - ref_time).total_seconds() / 86400.0 for dt in datetimes])  # giorni

# 3. Segnale regolare (sinusoide pulita)
true_period = 5.0  # giorni
signal = np.sin(2 * np.pi * times / true_period)

# 4. Lomb-Scargle
frequency, power = LombScargle(times, signal).autopower()
periods = 1 / frequency  # calcola i periodi corrispondenti alle frequenze


# 5. Plot con Plotly

# Primo grafico: segnale
fig_signal = go.Figure()
fig_signal.add_trace(go.Scatter(x=times, y=signal, mode='markers+lines', name='Segnale'))
fig_signal.update_layout(
    title='Segnale regolare (tempi irregolari)',
    xaxis_title='Tempo (giorni)',
    yaxis_title='Valore',
    template='plotly_white'
)

# Secondo grafico: Lomb-Scargle
fig_ls = go.Figure()
fig_ls.add_trace(go.Scatter(x=periods, y=power, mode='lines', name='Lomb-Scargle'))
fig_ls.add_trace(go.Scatter(
    x=[true_period], y=[max(power)],
    mode='markers',
    marker=dict(color='red', size=10),
    name=f'Periodo vero: {true_period} giorni'
))
fig_ls.update_layout(
    title='Trasformata Lomb-Scargle',
    xaxis_title='Periodo (giorni)',
    yaxis_title='Potenza',
    template='plotly_white'
)

# Mostra grafici
fig_signal.show()
fig_ls.show()

In [ ]:
# Create the date to separate the data
date_limit1 = pd.to_datetime('2023-09-25')
date_limit2 = pd.to_datetime('2024-01-01')

# Divide the data
df_pre_2024 = dataset[dataset['date_c'] < date_limit1]
df_2024 = dataset[dataset['date_c'] > date_limit2]

# Detrend the two dataset
detrended_pre = signal.detrend(df_pre_2024['deviation'])
detrended_2024 = signal.detrend(df_2024['deviation'])

# Create the figure of detrended values
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pre_2024['date_c'], y=detrended_pre, mode='lines+markers', name='Detrended pre-2024'))
fig.add_trace(go.Scatter(x=df_2024['date_c'], y=detrended_2024, mode='lines+markers', name='Detrended 2024'))
fig.update_layout(title='Detrended data', xaxis_title='Date', yaxis_title='Detrended value')
fig.show()

In [ ]:
# Initialize ChangeFinder object for change point detection
cf = changefinder.ChangeFinder()

# Extract measurement values and dates from the sorted DataFrame
values = qc_rows['deviation']
date = qc_rows['date_c']

# Define the starting point for calculating change scores (index in the 'values' list)
start_index = 50

# Initialize an empty list for scores
scores = [None] * len(values)  # Initialize with None for the initial period

# Warm-up the ChangeFinder with initial data points and calculate scores from the start_index
for i, v in enumerate(values):
    change_score = cf.update(v)
    if i >= start_index:
        scores[i] = change_score  # Assign the return value of update() to scores


# Create Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=date, y=scores, mode='lines', line=dict(color='gray')))
fig.update_layout(title='Change Score over Time (Starting from Index {})'.format(start_index),
                  xaxis_title='Date',
                  yaxis_title='Value')
fig.show()

In [1]:
for i in range (0,5):
    print(i)

0
1
2
3
4
